<a href="https://colab.research.google.com/github/FrankAlvaradoR/AI/blob/main/AlgoritmoGenetico_estados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import statistics
import sys
import time


def _generate_parent(length, geneSet, get_fitness):
    genes = []
    while len(genes) < length:
        sampleSize = min(length - len(genes), len(geneSet))
        genes.extend(random.sample(geneSet, sampleSize))
    fitness = get_fitness(genes)
    return Chromosome(genes, fitness)


def _mutate(parent, geneSet, get_fitness):
    childGenes = parent.Genes[:]
    index = random.randrange(0, len(parent.Genes))
    newGene, alternate = random.sample(geneSet, 2)
    childGenes[index] = alternate if newGene == childGenes[index] else newGene
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness)


def _mutate_custom(parent, custom_mutate, get_fitness):
    childGenes = parent.Genes[:]
    custom_mutate(childGenes)
    fitness = get_fitness(childGenes)
    return Chromosome(childGenes, fitness)


def get_best(get_fitness, targetLen, optimalFitness, geneSet, display,
             custom_mutate=None):
    if custom_mutate is None:
        def fnMutate(parent):
            return _mutate(parent, geneSet, get_fitness)
    else:
        def fnMutate(parent):
            return _mutate_custom(parent, custom_mutate, get_fitness)

    def fnGenerateParent():
        return _generate_parent(targetLen, geneSet, get_fitness)

    for improvement in _get_improvement(fnMutate, fnGenerateParent):
        display(improvement)
        if not optimalFitness > improvement.Fitness:
            return improvement


def _get_improvement(new_child, generate_parent):
    bestParent = generate_parent()
    yield bestParent
    while True:
        child = new_child(bestParent)
        if bestParent.Fitness > child.Fitness:
            continue
        if not child.Fitness > bestParent.Fitness:
            bestParent = child
            continue
        yield child
        bestParent = child


class Chromosome:
    def __init__(self, genes, fitness):
        self.Genes = genes
        self.Fitness = fitness


class Benchmark:
    @staticmethod
    def run(function):
        timings = []
        stdout = sys.stdout
        for i in range(100):
            sys.stdout = None
            startTime = time.time()
            function()
            seconds = time.time() - startTime
            sys.stdout = stdout
            timings.append(seconds)
            mean = statistics.mean(timings)
            if i < 10 or i % 10 == 9:
                print("{} {:3.2f} {:3.2f}".format(
                    1 + i, mean,
                    statistics.stdev(timings, mean) if i > 1 else 0))

In [9]:

import datetime
import unittest

class Rule:
    def __init__(self, node, adjacent):
        if node < adjacent:
            node, adjacent = adjacent, node
        self.Node = node
        self.Adjacent = adjacent

    def __eq__(self, other):
        return self.Node == other.Node and self.Adjacent == other.Adjacent

    def __hash__(self):
        return hash(self.Node) * 397 ^ hash(self.Adjacent)

    def __str__(self):
        return self.Node + " -> " + self.Adjacent

    def IsValid(self, genes, nodeIndexLookup):
        index = nodeIndexLookup[self.Node]
        adjacentNodeIndex = nodeIndexLookup[self.Adjacent]

        return genes[index] != genes[adjacentNodeIndex]

def load_data(localFileName):
    """ expects: T D1 [D2 ... DN]
        where T is the record type
        and D1 .. DN are record-type appropriate data elements
    """
    rules = set()
    nodes = set()
    with open(localFileName, mode='r') as infile:
        content = infile.read().splitlines()
    for row in content:
        if row[0] == 'e':  # e aa bb, aa and bb are node ids
            nodeIds = row.split(' ')[1:3]
            rules.add(Rule(nodeIds[0], nodeIds[1]))
            nodes.add(nodeIds[0])
            nodes.add(nodeIds[1])
            continue
        if row[0] == 'n':  # n aa ww, aa is a node id, ww is a weight
            nodeIds = row.split(' ')
            nodes.add(nodeIds[1])
    return rules, nodes


def build_rules(items):
    rulesAdded = {}
    for state, adjacent in items.items():
        for adjacentState in adjacent:
            if adjacentState == '':
                continue
            rule = Rule(state, adjacentState)
            if rule in rulesAdded:
                rulesAdded[rule] += 1
            else:
                rulesAdded[rule] = 1
    for k, v in rulesAdded.items():
        if v != 2:
            print("rule {} is not bidirectional".format(k))
    return rulesAdded.keys()


def get_fitness(genes, rules, stateIndexLookup):
    rulesThatPass = sum(1 for rule in rules
                        if rule.IsValid(genes, stateIndexLookup))
    return rulesThatPass


def display(candidate, startTime):
    timeDiff = datetime.datetime.now() - startTime
    print("{}\t{}\t{}".format(
        ''.join(map(str, candidate.Genes)),
        candidate.Fitness,
        timeDiff))


def main():
    colors = ["Orange", "Yellow", "Green", "Blue"]
    file = "/content/adjacent_states.col"

    rules, nodes = load_data(file)
    optimalValue = len(rules)
    colorLookup = {color[0]: color for color in colors}
    geneset = list(colorLookup.keys())
    startTime = datetime.datetime.now()
    nodeIndexLookup = {key: index for index, key in enumerate(sorted(nodes))}

    def fnDisplay(candidate):
      display(candidate, startTime)

    def fnGetFitness(genes):
      return get_fitness(genes, rules, nodeIndexLookup)

    best = get_best(fnGetFitness, len(nodes), optimalValue,geneset, fnDisplay)
    keys = sorted(nodes)
    for index in range(len(nodes)):
      print(keys[index] + " is " + colorLookup[best.Genes[index]])




if __name__ == '__main__':
  main()

OGYBBOYGGYBOBOYGBGYOYGBOYBOGYGOBGBYOGOYBOBYGGOYBYGO	82	0:00:00.000256
OGYBYOYGGYBOBOYGBGYOYGBOYBOGYGOBGBYOGOYBOBYGGOYBYGO	84	0:00:00.000878
OGYBYOYGGYBOBOYGBGYOYGBOYBOGYGOBGBYOGOYBOBOGGOYBYGO	85	0:00:00.000996
OGYBYOYGGYBOBOYGBGYOYGBOYBOGYGOBYBYOGOYBOBOGGOYBYGO	87	0:00:00.001179
OGYBYOYGGYBOBOYGBGYOYGBOYBOGYGOOYBGOGOYBOBOGGOYBYGO	90	0:00:00.001370
OGYBYOYGGYBOBOYYBGYOYGBOYBOGGGOOYBGOGOBBOBOGGOYBYGO	91	0:00:00.001831
OGYBYOYGGYBOGGYBBGYOYGBOYBOGGGOOYBGOGOBBOBOGGOYBYGO	93	0:00:00.002302
YGYBYOYGGYBOGGYBBGYOYYBOYBOGGGOOYBGYGOBBOBOGYOYBYGO	94	0:00:00.002909
YGYBYOYGGYBOGGYBBGYOYYOOYBOGGGOOYBGYGOBBOBOGYOYBYGO	95	0:00:00.003047
YGYBYOYGGYBOGGYBBGYOYYOOYBYGGGOOYBGYGOBBOBOGYOYBYGO	96	0:00:00.003254
YGYBYOYGGYBOGGYBBGOOYYOOYBYGGGOOYBGYGOBBOBOGYOYBYGO	97	0:00:00.003393
YGYBYOYGGYBOGGYBBGOOYYOOYBYGGGOOYBGYGOBBOBOGYBYBYGO	98	0:00:00.003630
YGYBYOYGGYBOGGYBBGOOYGOOOBYYGGOYYGGYGOBBGBOOYBYBYGO	99	0:00:00.004700
YGYBYOYGGYBOGGYBBGOOYGOOOBYYGGOYYGGYGOBBGBOOYBBBYOO	100	0:00:00.005421
OGYBYBYGGYBOGGYBBGO

In [8]:
file = "/content/adjacent_states.col"
rules, nodes = load_data(file)
print(rules)

{<__main__.Rule object at 0x7eff3fba1840>, <__main__.Rule object at 0x7eff3fba1cc0>, <__main__.Rule object at 0x7eff3fba0d90>, <__main__.Rule object at 0x7eff3fba3f10>, <__main__.Rule object at 0x7eff1fe05f90>, <__main__.Rule object at 0x7eff1fe064d0>, <__main__.Rule object at 0x7eff3fba3a90>, <__main__.Rule object at 0x7eff3fba1f00>, <__main__.Rule object at 0x7eff3fba1b10>, <__main__.Rule object at 0x7eff3fba3580>, <__main__.Rule object at 0x7eff3fba0ac0>, <__main__.Rule object at 0x7eff3fba37f0>, <__main__.Rule object at 0x7eff3fba3d90>, <__main__.Rule object at 0x7eff3fba3910>, <__main__.Rule object at 0x7eff3fba0130>, <__main__.Rule object at 0x7eff3fba1360>, <__main__.Rule object at 0x7eff3fba2e90>, <__main__.Rule object at 0x7eff3fba2620>, <__main__.Rule object at 0x7eff3fba22f0>, <__main__.Rule object at 0x7eff3fba15d0>, <__main__.Rule object at 0x7eff3fba2d10>, <__main__.Rule object at 0x7eff3fba2530>, <__main__.Rule object at 0x7eff3fba1000>, <__main__.Rule object at 0x7eff3f